In [7]:
!pip install matplotlib
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.datasets import cifar100
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import Dense, UpSampling2D, Dropout, BatchNormalization, RandomFlip, RandomTranslation, RandomRotation,RandomBrightness, RandomContrast, RandomZoom
from tensorflow.keras.callbacks import EarlyStopping

In [8]:
n_epoch = 1000 # Il y a le early stopping
batch_size = 100
taux_validation = 0.1
num_classes = 100
n_images = 50000 # Pour l'entrainement, et 10000 pour le test

In [9]:
def format_image(image, label):
    image = tf.image.resize(image, (224, 224)) / 255.0
    label = tf.squeeze(tf.one_hot(label, depth = num_classes), axis = 0)
    return  image, label

train_dataset, test_dataset = cifar100.load_data()

validation_size = int(n_images * taux_validation)
train_dataset = tf.data.Dataset.from_tensor_slices(train_dataset).map(format_image).shuffle(n_images)
test_dataset = tf.data.Dataset.from_tensor_slices(test_dataset).map(format_image)

validation_dataset = train_dataset.take(validation_size).batch(batch_size).prefetch(tf.data.AUTOTUNE)
train_dataset = train_dataset.skip(validation_size).batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [10]:
resnet = ResNet50V2(include_top=False, weights='imagenet', pooling="avg")

resnet.summary()

Model: "resnet50v2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, None,      │          0 │ -                 │
│ (InputLayer)        │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, None,      │          0 │ input_layer_2[0]… │
│ (ZeroPadding2D)     │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, None,      │      9,472 │ conv1_pad[0][0]   │
│                     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, None,      │          0 │ conv1_conv[0][0]  │
│ (ZeroPadding2D)     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, None,      │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_preac… │ (None, None,      │        256 │ pool1_pool[0][0]  │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_preac… │ (None, None,      │          0 │ conv2_block1_pre… │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, None,      │      4,096 │ conv2_block1_pre… │
│ (Conv2D)            │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, None,      │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, None,      │          0 │ conv2_block1_1_b… │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_pad  │ (None, None,      │          0 │ conv2_block1_1_r… │
│ (ZeroPadding2D)     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, None,      │     36,864 │ conv2_block1_2_p… │
│ (Conv2D)            │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, None,      │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, None,      │          0 │ conv2_block1_2_b… │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, None,      │     16,640 │ conv2_block1_pre… │
│ (Conv2D)            │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, None,      │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_out    │ (None, None,      │          0 │ conv2_block1_0_c

 Total params: 23,564,800 (89.89 MB)

 Trainable params: 23,519,360 (89.72 MB)

 Non-trainable params: 45,440 (177.50 KB)

In [11]:
model = Sequential([
    RandomFlip("horizontal"),
    RandomTranslation(0.2,0.2),
    RandomRotation(0.2),
    RandomZoom(0.2),
    RandomBrightness(0.2,value_range=(0,1)),
    RandomContrast(0.2),
    resnet,
    Dropout(0.25),
    Dense(256, activation="sigmoid", kernel_regularizer = tf.keras.regularizers.L1(0.001)),
    Dropout(0.5),
    Dense(num_classes, activation="softmax", kernel_regularizer = tf.keras.regularizers.L2(0.001))
])

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
history = model.fit(train_dataset, epochs=n_epoch, batch_size=batch_size, validation_data = validation_dataset, callbacks = early_stopping_callback)
model.save("~/modele_enseignant.keras")

Epoch 1/1000
450/450 ━━━━━━━━━━━━━━━━━━━━ 831s 2s/step - accuracy: 0.0439 - loss: 7.9252 - val_accuracy: 0.0736 - val_loss: 4.9192
Epoch 2/1000
450/450 ━━━━━━━━━━━━━━━━━━━━ 787s 2s/step - accuracy: 0.1104 - loss: 4.5712 - val_accuracy: 0.1470 - val_loss: 4.3087
Epoch 3/1000
450/450 ━━━━━━━━━━━━━━━━━━━━ 786s 2s/step - accuracy: 0.1559 - loss: 4.2471 - val_accuracy: 0.1260 - val_loss: 4.8950
Epoch 4/1000
450/450 ━━━━━━━━━━━━━━━━━━━━ 786s 2s/step - accuracy: 0.1954 - loss: 4.0376 - val_accuracy: 0.2724 - val_loss: 3.5899
Epoch 5/1000
450/450 ━━━━━━━━━━━━━━━━━━━━ 786s 2s/step - accuracy: 0.2358 - loss: 3.8622 - val_accuracy: 0.3182 - val_loss: 3.4123
Epoch 6/1000
450/450 ━━━━━━━━━━━━━━━━━━━━ 783s 2s/step - accuracy: 0.2723 - loss: 3.7094 - val_accuracy: 0.3534 - val_loss: 3.2817
Epoch 7/1000
450/450 ━━━━━━━━━━━━━━━━━━━━ 784s 2s/step - accuracy: 0.2995 - loss: 3.6020 - val_accuracy: 0.3832 - val_loss: 3.1347
Epoch 8/1000
179/450 ━━━━━━━━━━━━━━━━━━━━ 7:37 2s/step - accuracy: 0.3300 - loss: 3

In [ ]:
plt.plot(history.history['accuracy'][:21])
plt.plot(history.history['val_accuracy'][:21])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
model.evaluate(test)

In [ ]:
!mc cp ~/modele_enseignant.keras s3/afeldmann/modele_enseignant.keras